In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchaudio.transforms as T
import torchvision.models as models
import torchaudio
import os
from torch.utils.data import Dataset, DataLoader
import random
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler
# Implement Stratified K-Folds Cross-validation
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

import config

In [2]:
if config.ADSMI:
    from DL_finetune import ADSMI_DL_CV as DL

## Models


In [3]:
print(config.channels)

class Resnet50_Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Resnet50_Classifier, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.resnet50.conv1 = nn.Conv2d(config.channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet50(x)
    



class ResNet101_Classifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNet101_Classifier, self).__init__()
        self.resnet101 = models.resnet101(pretrained=True)
        # Modify the input layer to match your input data channels
        # If your input data has different channels, adjust this line accordingly
        self.resnet101.conv1 = nn.Conv2d(config.channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_features = self.resnet101.fc.in_features
        self.resnet101.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet101(x)
    

    
class ModifiedResnet50_Classifier(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResnet50_Classifier, self).__init__()
        
        # Load the pretrained ResNet-50 model
        self.resnet50 = models.resnet50(pretrained=True)
        
        # Modify the first convolutional layer
        self.resnet50.conv1 = nn.Conv2d(config.channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Remove the final fully connected layer
        self.resnet50.fc = nn.Identity()  # Set the final layer to an identity mapping
        
        # Define the custom fully connected layers
        num_features = 2048 
        self.fc1 = nn.Linear(num_features, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.resnet50(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

3


In [4]:
#------Datasplit
# Load the dataframe
labels_file = pd.read_csv('./data/labeled_ADSMI/labels_int.csv', index_col=0)
train_df, val_df = train_test_split(labels_file, test_size=0.1, stratify=labels_file['Label_int'], random_state=42)
# train test split
print("Train size: ", len(train_df))
print("Val size: ", len(val_df))




#------Data fold generation for cross-validation
n_folds = 8
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)


Train size:  4248
Val size:  472


In [5]:


num_classes = len(set(labels_file["Label_int"]))  # Assuming the number of classes is the unique count of "Label_int" in your labels_file
#model = Resnet50_Classifier(num_classes)
model = ModifiedResnet50_Classifier(num_classes)
#model = ResNet101_Classifier(num_classes)



#  Transfer the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0004, weight_decay = 1e-4 ) # Adjust the value as needed)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.8, verbose=True)

train_losses = []
test_losses = []
config.best_accuracy = 0
config.model_path = "./results_standalone/test2_checkpoint.pth"


for fold, (train_indices, test_indices) in enumerate(skf.split(train_df, train_df['Label_int'])):
    
    train_loader, test_loader = DL.create_generators_finetune(train_df=train_df,train_indices=train_indices, test_indices=test_indices)
    #  Create an instance of the model
    
    # Training loop
    print("Fold: ", fold)
    num_epochs = 20  # Adjust this as needed
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (spectrograms, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
            spectrograms = spectrograms.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(spectrograms)
            loss = criterion(outputs, labels)
            train_losses.append(loss.item())
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()



        correct_predictions = 0
        total_samples = 0

        # Set the model to evaluation mode (important for dropout and batch normalization)
        model.eval()

        # Iterate through the test set
        with torch.no_grad():  # Disable gradient computation during testing
            for spectrograms, labels in test_loader:
                # Move data to the testing device
                spectrograms = spectrograms.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(spectrograms)
                loss = criterion(outputs, labels)
                test_losses.append(loss.item())
                
                # Compute the predicted labels
                _, predicted = torch.max(outputs, 1)

                # Update evaluation metrics
                total_samples += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

        #if new test accuracy is better than the previous best, save the model
        if correct_predictions / total_samples > config.best_accuracy:
            config.best_accuracy = correct_predictions / total_samples
            torch.save(model, config.model_path)
            
        # Step the learning rate scheduler
        scheduler.step(test_losses[-1])

        # Calculate accuracy or other evaluation metrics
        accuracy = correct_predictions / total_samples
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        print(f"Test Accuracy: {accuracy * 100:.2f}%")

c:\Users\Gabriel\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Gabriel\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda
Fold:  0


100%|██████████| 133/133 [00:42<00:00,  3.13it/s]


Epoch [1/20], Loss: 0.6067
Test Accuracy: 69.70%


100%|██████████| 133/133 [00:40<00:00,  3.27it/s]


...

Epoch [7/20], Loss: 0.5755
Test Accuracy: 88.54%


100%|██████████| 133/133 [00:39<00:00,  3.34it/s]


Epoch [8/20], Loss: 0.3719
Test Accuracy: 89.31%


 32%|███▏      | 42/133 [00:13<00:28,  3.19it/s]


KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
config.desired_length_in_seconds = 15

# Set the device for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a data loader for the test set
val_loader = DL.create_generators_finetune_val(val_df=val_df,)  # Assuming create_generators_finetune returns (train_loader, test_loader)


#model = torch.load(config.model_path)
# Transfer the model to the testing device
model.to(device)

# Define a criterion for evaluation (e.g., cross-entropy loss for classification)
criterion = nn.CrossEntropyLoss()

# Initialize variables for evaluation metrics (e.g., accuracy)
correct_predictions = 0
total_samples = 0

# Define the label dictionary
true_labels_dic = {0: '[Kreischen]', 1: '[Kreischen][Quietschen]', 2: '[Negativ]', 3: '[Quietschen]'}

# Set the model to evaluation mode (important for dropout and batch normalization)
model.eval()

# Initialize lists to store all true labels and predicted labels
all_true_labels = []
all_predicted_labels = []

# Iterate through the test set
with torch.no_grad():
    for spectrograms, labels in val_loader:
        # Move data to the testing device
        spectrograms = spectrograms.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(spectrograms)
        
        # Compute the predicted labels
        _, predicted = torch.max(outputs, 1)

        # Append true and predicted labels to the lists
        all_true_labels.extend(labels.cpu().numpy())
        all_predicted_labels.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays
all_true_labels = np.array(all_true_labels)
all_predicted_labels = np.array(all_predicted_labels)

# Calculate accuracy, precision, recall, F1-score, etc. using all_true_labels and all_predicted_labels
accuracy = np.mean(all_true_labels == all_predicted_labels)
precision, recall, f1_score, _ = precision_recall_fscore_support(all_true_labels, all_predicted_labels, average='weighted')
_, recall_per_class, _, _ = precision_recall_fscore_support(all_true_labels, all_predicted_labels)



#calculate balanced accuracy
balanced_accuracy = np.mean(recall_per_class)
print(f"Balanced Accuracy: {balanced_accuracy * 100:.2f}%")

print(f"Val Accuracy: {accuracy * 100:.2f}%")
print(f"\nEvaluation Results:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
conf_mat = confusion_matrix(all_true_labels, all_predicted_labels)
def plot_confusion_matrix(conf_mat, class_labels):
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    
    # Convert class_labels to a list of strings (or integers)
    class_labels = [str(label) for label in class_labels]
    
    ax = sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", cbar=False,
                xticklabels=class_labels,
                yticklabels=class_labels)
    
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.xticks(rotation=90)  # Rotate x-axis labels vertically
    plt.show()

plot_confusion_matrix(conf_mat, true_labels_dic.values())



NameError: name 'config' is not defined